<a href="https://colab.research.google.com/github/luizmv8080/aprendizaje-profundo/blob/main/Tarea1/Ejercicio_3_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Regresion lineal con Pytorch
## Red neuronal a medio nivel

In [196]:
import os
import random
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

In [197]:
URL = 'https://raw.githubusercontent.com/gibranfp/CursoAprendizajeProfundo/2023-1/data/califs/califs.csv'
data_dir = '../data'
filepath = os.path.join(data_dir, 'califs.csv')

def set_seed(seed=0):
    """Initializes pseudo-random number generators."""
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)

! mkdir {data_dir}
! wget -nc {URL} -O {filepath}

mkdir: cannot create directory ‘../data’: File exists
File ‘../data/califs.csv’ already there; not retrieving.


In [198]:
df = pd.read_csv(filepath)
df.head(2)

,prev,horas,calif
0,5.6,8.2,5.1
1,6.7,9.3,5.9


In [199]:
x_trn = np.array(df.iloc[:,:2], dtype="float32")
y_trn = np.array(df.iloc[:,-1], dtype="float32")[..., np.newaxis]

x_trn = torch.tensor(x_trn)
y_trn = torch.tensor(y_trn)

In [200]:
ds = TensorDataset(x_trn, y_trn)

In [201]:
def build_dl(batch_size=16, shuffle=True):
    return DataLoader(ds, batch_size=batch_size, shuffle=True)

dl = build_dl()

In [202]:
def train(model, dl, lr, epochs=10):

    opt = optim.SGD(model.parameters(), lr=lr)
    loss_hist = []

    for epoch in range(epochs):

        loss_hist = []
        
        for x, y_true in dl:
  
            y_lgts = model(x)

            loss = F.mse_loss(y_lgts, y_true)

            opt.zero_grad()

            loss.backward()

            opt.step()

            loss_hist.append(loss.item() * 100)
        loss = np.mean(loss_hist)

        if epoch == (epochs - 1):
          print(f'E{epoch:02} loss=[{loss:6.2f}]')


        
def train_model(build_model, lr, epochs=1000):
    set_seed()
    dl = build_dl()
    model = build_model
    train(model, dl, lr, epochs)

### Contruyendo modelo a medio nivel con dos capas ocultas

In [204]:
#se define la clase RegLin que hereda de torch.nn.Module
class LinRegMedTwo(nn.Module):

    #se define el inicializador
    def __init__(self,capa_uno,capa_dos):
        # se llama al inicializador de la clase padre
        super().__init__()
        # se definen las capas como atributos de la clase
        self.fc1 = nn.Linear(2, capa_uno)
        self.fc2 = nn.Linear(capa_uno, capa_dos)
        self.fc3 = nn.Linear(capa_dos, 1)

    # método para inferencia
    def forward(self, x):
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        x = F.relu(x)
        x = self.fc3(x)
        x = F.relu(x)
        return x

def build_med_two(capa_uno,capa_dos):
    return LinRegMedTwo(capa_uno,capa_dos)


In [141]:
numero_capas = [11,10,6,4,2]
learning_rate = [6, 4, 2]

for i in numero_capas:
  for j in learning_rate:
    med_two = build_med_two(i,j)
    print("Para un numero de neuronas de. {0} en la primera capa y un lr de {1} se tiene una perdida de:".format(i,j))
    train_model(med_two, 0.0001)
  print("=====================================================")



Para un numero de neuronas de. 11 en la primera capa y un lr de 6 se tiene una perdida de:
E999 loss=[5668.87]
Para un numero de neuronas de. 11 en la primera capa y un lr de 4 se tiene una perdida de:
E999 loss=[ 20.00]
Para un numero de neuronas de. 11 en la primera capa y un lr de 2 se tiene una perdida de:
E999 loss=[1202.21]
Para un numero de neuronas de. 10 en la primera capa y un lr de 6 se tiene una perdida de:
E999 loss=[ 19.71]
Para un numero de neuronas de. 10 en la primera capa y un lr de 4 se tiene una perdida de:
E999 loss=[5668.87]
Para un numero de neuronas de. 10 en la primera capa y un lr de 2 se tiene una perdida de:
E999 loss=[ 20.87]
Para un numero de neuronas de. 6 en la primera capa y un lr de 6 se tiene una perdida de:
E999 loss=[ 18.90]
Para un numero de neuronas de. 6 en la primera capa y un lr de 4 se tiene una perdida de:
E999 loss=[5668.87]
Para un numero de neuronas de. 6 en la primera capa y un lr de 2 se tiene una perdida de:
E999 loss=[1107.81]
Para un 

In [205]:
med_two = build_med_two(6,6)
train_model(med_two,0.002)

E999 loss=[ 18.38]


In [206]:
lista_parametros = list(med_two.named_parameters())

In [207]:
#lista_parametros

In [208]:
def redNeuronalDos(x,w1,w2,w3,b1,b2,b3):
  
  X = x[:,np.newaxis]
  z1 = np.dot(w1,X) + b1.T
  a1 = np.maximum(0,z1)
  z2 = np.dot(w2,a1) + b2.T
  a2 = np.maximum(0,z2)
  z3 = np.dot(w3,a2) + b3
  pred = np.maximum(0,z3)
  return pred.item()

In [209]:
w1 = lista_parametros[0][1].detach().numpy()
b1 = np.array([lista_parametros[1][1].detach().numpy()])
w2 = lista_parametros[2][1].detach().numpy()
b2 = np.array([lista_parametros[3][1].detach().numpy()])
w3 = lista_parametros[4][1].detach().numpy()
b3 = lista_parametros[5][1].detach().numpy().item()

y_prediccion_dos = []
for i in range(x_trn.shape[0]):
  y_prediccion_dos.append(redNeuronalDos(x_trn[i],w1,w2,w3,b1,b2,b3))

In [190]:
x_trn[0]

tensor([5.6000, 8.2000])

In [210]:
df_rdc = df.copy()
df_rdc['Prediccion'] = y_prediccion_dos
df_rdc.head(3)

,prev,horas,calif,Prediccion
0,5.6,8.2,5.1,5.475646
1,6.7,9.3,5.9,6.301627
2,5.1,10.0,5.4,6.275753


### Contruyendo modelo a medio nivel con una capa oculta

In [234]:
class LinRegMedOne(nn.Module):

    #se define el inicializador
    def __init__(self,n_neuronas):
        # se llama al inicializador de la clase padre
        super().__init__()
        # se definen las capas como atributos de la clase
        self.fc1 = nn.Linear(2, n_neuronas)
        self.fc2 = nn.Linear(n_neuronas, 1)

    # método para inferencia
    def forward(self, x):
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        x = F.relu(x)
        return x

def build_med_one(n_neuronas):
    return LinRegMedOne(n_neuronas)

In [215]:
numero_neuronas = [11,10,6,4,2]


for i in numero_neuronas:
  med_one = build_med_one(i)
  print("Para un numero de neuronas de. {0} en la primera capa se tiene una perdida de:".format(i))
  train_model(med_one, 0.0001)
  print("=====================================================")

Para un numero de neuronas de. 11 en la primera capa se tiene una perdida de:
E999 loss=[5668.87]
Para un numero de neuronas de. 10 en la primera capa se tiene una perdida de:
E999 loss=[ 18.85]
Para un numero de neuronas de. 6 en la primera capa se tiene una perdida de:
E999 loss=[ 21.25]
Para un numero de neuronas de. 4 en la primera capa se tiene una perdida de:
E999 loss=[ 24.94]
Para un numero de neuronas de. 2 en la primera capa se tiene una perdida de:
E999 loss=[1118.82]


In [235]:
med_one = build_med_one(10)
train_model(med_one, 0.002)

E999 loss=[ 18.43]


In [236]:
lista_parametros_dos = list(med_one.named_parameters())

In [240]:
def redNeuronalUno(x,w1,w2,b1,b2):
  
  X = x[:,np.newaxis]
  z = np.dot(w1,X) + b1.T
  a1 = np.maximum(0,z)
  z2 = np.dot(w2,a1) + b2
  pred = np.maximum(0,z2)

  return pred.item()

In [241]:
w1_1 = lista_parametros_dos[0][1].detach().numpy()
b1_1 = np.array([lista_parametros_dos[1][1].detach().numpy()])
w2_1 = lista_parametros_dos[2][1].detach().numpy()
b2_1 = lista_parametros_dos[3][1].detach().numpy().item()

y_prediccion_dos = []
for i in range(x_trn.shape[0]):
  y_prediccion_dos.append(redNeuronalUno(x_trn[i],w1_1,w2_1,b1_1,b2_1))

In [242]:
df_rdc = df.copy()
df_rdc['Prediccion'] = y_prediccion_dos
df_rdc.head(3)

,prev,horas,calif,Prediccion
0,5.6,8.2,5.1,5.405046
1,6.7,9.3,5.9,6.256642
2,5.1,10.0,5.4,6.186957


# Prediccion para un alumno que estudió durante 12 horas para el examen y en un examen previo obtuvo 3 de calificación.

### Para nuestra red con una sola capa se tiene:

In [245]:
redNeuronalUno(np.array([3,12]),w1_1,w2_1,b1_1,b2_1)

6.672335704008775

### Para nuestra red con dos capas se tiene:

In [247]:
redNeuronalDos(np.array([3,12]),w1,w2,w3,b1,b2,b3)

6.559881507798183

# Resultados de construccion de red neuronal de medio nivel

### Con una red de una sola capa oculta

Se creo una red con una sola capa oculta. Esa capa oculta cuenta con 10 neuronas. Se escogio un **learning rate** de **0.002** ya que minimizaba la perdida a un **18.43%**.

Respecto a la prediccion de un estudiante que estudió durante 12 horas para el examen y en un examen previo obtuvo 3 de calificación, se obtuvo que sacaria una calificacion de **6.67**.

### Con red de dos capas ocultas

Se creo una red con dos capas ocultas. La segunda capa oculta solo contaba con 6 neuronas, mientras que la primera capa oculta conto con 6 neuronas. Se escogio un **learning rate** de **0.002** ya que minimizaba la perdida a un **18.38%**.

Respecto a la prediccion de un estudiante que estudió durante 12 horas para el examen y en un examen previo obtuvo 3 de calificación, se obtuvo que sacaria una calificacion de **6.55**. Vemos que ambas predicciones se mantienen en el rango.